In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import image_dataset_from_directory
from tensorflow import keras
from keras import layers
from keras import models
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [ ]:
train_dir = '../Generator/dataset/train'
test_dir = '../Generator/dataset/test'

train_square_dir = train_dir + '/square'
train_circle_dir = train_dir + '/circle'
train_none_dir = train_dir + '/none'
train_circle_square_dir = train_dir + '/square_circle'
test_circle_square_dir = test_dir + '/square_and_circle'

In [ ]:
IMG_SIZE = 500
train_datagen = ImageDataGenerator(
    #rescale=1./255,  # Normalize pixel values
    #rotation_range=20,  # Randomly rotate images up to 20 degrees
    #width_shift_range=0.2,  # Shift images horizontally up to 20%
    #height_shift_range=0.2,  # Shift images vertically up to 20%
    #shear_range=0.2,  # Shear images randomly
    #zoom_range=0.2,  # Zoom images randomly
    #horizontal_flip=True  # Flip images horizontally
)

train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to training directory
    target_size=(IMG_SIZE, IMG_SIZE),  # Set your image dimensions
    batch_size=32,  # Adjust batch size as needed
    class_mode='categorical'  # One-hot encode labels for multi-class classification
)

In [ ]:
IMG_SIZE = 500
test_datagen = ImageDataGenerator(
    #rescale=1./255,  # Normalize pixel values
    #rotation_range=20,  # Randomly rotate images up to 20 degrees
    #width_shift_range=0.2,  # Shift images horizontally up to 20%
    #height_shift_range=0.2,  # Shift images vertically up to 20%
    #shear_range=0.2,  # Shear images randomly
    #zoom_range=0.2,  # Zoom images randomly
    #horizontal_flip=True  # Flip images horizontally
)

test_generator = test_datagen.flow_from_directory(
    test_dir,  # Path to training directory
    target_size=(IMG_SIZE, IMG_SIZE),  # Set your image dimensions
    batch_size=32,  # Adjust batch size as needed
    class_mode='categorical'  # One-hot encode labels for multi-class classification
)

In [ ]:
train_dataset = image_dataset_from_directory(train_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=32)
test_dataset = image_dataset_from_directory(test_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=32)

In [ ]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

In [ ]:
history = model.fit(train_generator, epochs=10, batch_size=64)

In [ ]:
model.save('models/shape_classifier.h5')

In [ ]:
img = np.array([])
for data_batch, label in test_dataset.take(1):
        image = data_batch[1].numpy().astype("uint8")
        img = np.append(img, image)
        plt.imshow(image)
        plt.show()
        break
img = img.reshape(IMG_SIZE, IMG_SIZE, 3)
img.astype("float32") / 255
img = np.expand_dims(img, axis=0)
print(img.shape)
print(model.predict(img).round())

val_loss, val_acc = model.evaluate(train_generator, batch_size=32)
print(val_loss, val_acc)


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
img = tf.keras.preprocessing.image.load_img('cats_and_dogs_small/validation/dogs/dog.1010.jpg', target_size=(150, 150), interpolation='bilinear')

In [ ]:
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
print(img_array.shape)
result = model.predict(img_array)
print("Result: ", result.round())